In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from typing import List, Tuple, Dict, Union, Any, Optional
from tqdm import tqdm
import json
try:
    os.chdir(os.path.join(os.getcwd(), '../../Language-Technology-Project'))
    print(os.getcwd())
except:
    print("ALready in current dir")

c:\Users\mo\Documents\Organisations\RUG\Language-Technology-Project


### Load the data


In [2]:
path = "./datasets/threads.jsonl"

def read_data(path: str) -> List[dict]:
    # parse the content of the json file
    text = open(path, 'r').read()
    # parse jsonl
    data = [json.loads(jline) for jline in text.split('\n') if jline]
    return data

data = read_data(path)

In [3]:
data_light = data[:1024]

In [5]:
def write_data(data: List[dict], path: str) -> None:
    with open(path, 'w') as outfile:
        for entry in data:
            json.dump(entry, outfile)
            outfile.write('\n')

write_data(data_light, "./datasets/threads_light.jsonl")